# Importações

In [1]:
import tensorflow as tf
import requests
import pandas as pd
import numpy as np

import json
import math

2023-07-17 18:00:00.998861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 18:00:06.234964: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-17 18:00:06.235061: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-17 18:00:33.510440: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Globais

In [2]:
global experimentoCorrente
global diaCorrente
global modelo
global dfDados
global X

# Obtendo Experimentos

In [8]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExperimentoPreRedeTreinadas/listaParaScore"

r = requests.get(url)
j = r.json()
dfExperimento = pd.DataFrame.from_dict(j)
dfExperimento.head()

,id,nome,experimentoPreRedeId,grupoAcaoId,regraProjecaoId,geraScore,periodoTreinoRedeId,nomeArquivoRede,qtdeDias1,qtdeDias2
0,1,01,1,12,127,S,7,ExperimentoSintetico_Prometeu_v0_aInicial,20,None


# Trata Experimento

In [10]:
experimentoCorrente = dfExperimento.iloc[0]
nome_arquivo = '/root/executa-rede/redes/' + experimentoCorrente.nomeArquivoRede + '.h5'
# carrega modelo
modelo = tf.keras.models.load_model(nome_arquivo)
urlDias = "http://vps-40d69db1.vps.ovh.ca:23004/api/DiaPregaos/obtemPeriodoPrevisaoTeste?idPeriodoTreinoRede={}".format(experimentoCorrente.periodoTreinoRedeId)
print(urlDias)
r = requests.get(urlDias)
j = r.json()
# carrega dfDias
dfDias = pd.DataFrame.from_dict(j)
print(dfDias.head())


http://vps-40d69db1.vps.ovh.ca:23004/api/DiaPregaos/obtemPeriodoPrevisaoTeste?idPeriodoTreinoRede=7
                       data  mes   ano    diaNum     dataStr dataDbStr
0  2023-04-03T00:00:00.000Z    4  2023  20230403  03-04-2023      None
1  2023-04-04T00:00:00.000Z    4  2023  20230404  04-04-2023      None
2  2023-04-05T00:00:00.000Z    4  2023  20230405  05-04-2023      None
3  2023-04-06T00:00:00.000Z    4  2023  20230406  06-04-2023      None
4  2023-04-10T00:00:00.000Z    4  2023  20230410  10-04-2023      None


# Trata Dia

In [12]:
diaCorrente = dfDias.iloc[0]
urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemPorDiaNum?diaNum={}&qtdeDias={}".format(diaCorrente.diaNum,experimentoCorrente.qtdeDias1)
print(urlExemplos)
r = requests.get(urlExemplos)
j = r.json()

# meu dados totais de entrada
dfDados = pd.DataFrame.from_dict(j)
print(dfDados.head())
dfTratadoX = dfDados.apply(lambda linha: np.fromstring(linha['campoX'], dtype=float, sep=','), axis='columns')
print(dfTratadoX.head())

# meu dataframe X
X = np.stack(dfTratadoX.values)
X.shape

http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemPorDiaNum?diaNum=20230403&qtdeDias=20
                                              campoX ticker  diaNumPrevisao
0   0.925614, 0.926316, 0.926316, 0.931930, 0.931...  ABEV3        20230403
1   0.957627, 0.957627, 0.991525, 1.025424, 1.025...  AERI3        20230403
2   1.114391, 1.100861, 1.084871, 1.099631, 1.107...  ALPA4        20230403
3   0.947429, 0.946286, 0.943429, 0.951429, 0.950...  ALSO3        20230403
4   0.953125, 0.937500, 0.937500, 0.937500, 0.953...  AMAR3        20230403
0    [0.925614, 0.926316, 0.926316, 0.93193, 0.9319...
1    [0.957627, 0.957627, 0.991525, 1.025424, 1.025...
2    [1.114391, 1.100861, 1.084871, 1.099631, 1.107...
3    [0.947429, 0.946286, 0.943429, 0.951429, 0.950...
4    [0.953125, 0.9375, 0.9375, 0.9375, 0.953125, 0...
dtype: object


(83, 300)

# Previsões

In [13]:
result = modelo.predict(X)

2023-07-18 12:58:14.058633: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2457600 exceeds 10% of free system memory.
2023-07-18 12:58:14.108253: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2457600 exceeds 10% of free system memory.
2023-07-18 12:58:14.302948: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2457600 exceeds 10% of free system memory.
2023-07-18 12:58:14.479867: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1228800 exceeds 10% of free system memory.
2023-07-18 12:58:14.481354: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1228800 exceeds 10% of free system memory.


3/3 [==============================] - 20s 198ms/step


In [21]:
print(type(result))

<class 'numpy.ndarray'>


# Colocando os resultados em um dataframe

In [16]:
dfDados['resultado'] = result

In [17]:
dfDados.head()

,campoX,ticker,diaNumPrevisao,resultado
0,"0.925614, 0.926316, 0.926316, 0.931930, 0.931...",ABEV3,20230403,0.001159
1,"0.957627, 0.957627, 0.991525, 1.025424, 1.025...",AERI3,20230403,0.001159
2,"1.114391, 1.100861, 1.084871, 1.099631, 1.107...",ALPA4,20230403,0.001159
3,"0.947429, 0.946286, 0.943429, 0.951429, 0.950...",ALSO3,20230403,0.001159
4,"0.953125, 0.937500, 0.937500, 0.937500, 0.953...",AMAR3,20230403,0.001159


In [27]:
dfPrevisao = pd.DataFrame({'score': pd.Series(result.flatten())})

In [35]:
dfPrevisao.head()

,score,diaNum,experimentoPreRedeTreinadaId,ticker
0,0.001159,20230403,1,ABEV3
1,0.001159,20230403,1,AERI3
2,0.001159,20230403,1,ALPA4
3,0.001159,20230403,1,ALSO3
4,0.001159,20230403,1,AMAR3


In [29]:
dfPrevisao['diaNum'] = diaCorrente.diaNum

In [32]:
dfPrevisao['experimentoPreRedeTreinadaId'] = experimentoCorrente.id

In [34]:
dfPrevisao['ticker'] = dfDados.ticker

# Enviar Dados de Previsão

In [41]:
print('ticker previsão:' , len(dfPrevisao))
json = dfPrevisao.to_dict(orient='list')
urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExperimentoPreRedeTreinadaScores/recebePrevisaoPreRedeTreinada'
#print(json)
response = requests.post(urlPrevisao, data=json)
print(response)

ticker previsão: 83
<Response [200]>


In [38]:
print(json)

{'score': [0.00115868984721601, 0.0011586915934458375, 0.0011586170876398683, 0.0011586655164137483, 0.0011588113848119974, 0.0011587709886953235, 0.8213529586791992, 0.001158739672973752, 0.0011587947374209762, 0.0011587454937398434, 0.0011587826302275062, 0.0011586116161197424, 0.0011587319895625114, 0.0011588080087676644, 0.0011587390908971429, 0.0011586910113692284, 0.0011587280314415693, 0.8212597370147705, 0.0011586942709982395, 0.8213174939155579, 0.00115875992923975, 0.0011587191838771105, 0.001158692641183734, 0.0011586705222725868, 0.0011586854234337807, 0.0011587693588808179, 0.0011587428161874413, 0.0011586485197767615, 0.0011587543413043022, 0.8213072419166565, 0.0011587163899093866, 0.0011586948530748487, 0.0011587119661271572, 0.8213642835617065, 0.0011587516637519002, 0.0011586584150791168, 0.0011587009066715837, 0.0011586904292926192, 0.0011587119661271572, 0.0011587064946070313, 0.001158747705630958, 0.0011587119661271572, 0.001158760511316359, 0.0011586336186155677, 